### 📌 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%matplotlib inline

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import tensorflow as tf
import os
import re

In [ ]:
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.models import Model
import tensorflow.keras

os.chdir('/content/drive/MyDrive/클라우드 플랫폼/팀 프로젝트 (클플)/Final Data')

input_shape = (224, 224, 3)

mobilenet_model = mobilenet_v2.MobileNetV2(include_top=False, weights='imagenet', 
                                              input_shape=input_shape)

for i, layer in enumerate(mobilenet_model.layers):
   print(i, layer.name) 

0 input_3
1 Conv1
2 bn_Conv1
3 Conv1_relu
4 expanded_conv_depthwise
5 expanded_conv_depthwise_BN
6 expanded_conv_depthwise_relu
7 expanded_conv_project
8 expanded_conv_project_BN
9 block_1_expand
10 block_1_expand_BN
11 block_1_expand_relu
12 block_1_pad
13 block_1_depthwise
14 block_1_depthwise_BN
15 block_1_depthwise_relu
16 block_1_project
17 block_1_project_BN
18 block_2_expand
19 block_2_expand_BN
20 block_2_expand_relu
21 block_2_depthwise
22 block_2_depthwise_BN
23 block_2_depthwise_relu
24 block_2_project
25 block_2_project_BN
26 block_2_add
27 block_3_expand
28 block_3_expand_BN
29 block_3_expand_relu
30 block_3_pad
31 block_3_depthwise
32 block_3_depthwise_BN
33 block_3_depthwise_relu
34 block_3_project
35 block_3_project_BN
36 block_4_expand
37 block_4_expand_BN
38 block_4_expand_relu
39 block_4_depthwise
40 block_4_depthwise_BN
41 block_4_depthwise_relu
42 block_4_project
43 block_4_project_BN
44 block_4_add
45 block_5_expand
46 block_5_expand_BN
47 block_5_expand_relu
48 b

In [ ]:
mobilenet_model.trainable = True

for layer in mobilenet_model.layers[:36]:
  layer.trainable = False

mobilenet_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [ ]:
len(mobilenet_model.trainable_variables)

120

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=30, 
                                  # shear_range=5.5,  
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  zoom_range=0.1,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='reflect')

train_generator = train_datagen.flow_from_directory('./model_data/혼합_자세/train',
                                                    target_size=(224, 224),       #코드 변경
                                                    batch_size=5,
                                                    classes = ['good', 'bad'],    # 코드 추가
                                                    class_mode='binary')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('./model_data/혼합_자세/test',
                                                    target_size=(224, 224),       #코드 변경
                                                    batch_size=5,
                                                    classes = ['good', 'bad'],    # 코드 추가
                                                    class_mode='binary')

Found 448 images belonging to 2 classes.
Found 152 images belonging to 2 classes.


In [ ]:
print(train_generator.class_indices)

{'good': 0, 'bad': 1}


## ✔️ 일반 Model

### 👉 epoch : 100 기준으로 돌리기 (model1)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

model1 = Sequential()
model1.add(mobilenet_model)
model1.add(Flatten())       
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=2e-5),
              metrics=['accuracy'])

model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten_2 (Flatten)         (None, 62720)             0         
                                                                 
 dense_2 (Dense)             (None, 1)                 62721     
                                                                 
Total params: 2,320,705
Trainable params: 2,260,801
Non-trainable params: 59,904
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
tf.random.set_seed(100)

batch_size = 20
num_classes = 2
epochs = 200    #EarlyStopping 적용(안돌아가면 30회)
input_shape = (224, 224, 3)   #코드 변경

MODEL_DIR = './model/'
if not os.path.exists('./model/'):
    os.mkdir('./model/')

modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

history = model1.fit(train_generator, 
                     epochs=epochs,
                     validation_data=test_generator, #validation_steps=4,
                     verbose=1, callbacks=[early_stopping_callback,checkpointer])

patience = 20

print("\n-----------------------------")
print(f"Training Accuracy: {round(history.history['accuracy'][-1 * patience -1], 4)}")
print(f"Training Loss: {round(history.history['loss'][-1 * patience - 1], 4)}")
print(f"Validation Accuracy: {round(history.history['val_accuracy'][-1 * patience - 1], 4)}")
print(f"Validation Loss: {round(history.history['val_loss'][-1 * patience - 1], 4)}")

Epoch 1/200
90/90 [==============================] - ETA: 0s - loss: 0.9569 - accuracy: 0.5335
Epoch 1: val_loss improved from inf to 0.77938, saving model to ./model/01-0.7794.hdf5
90/90 [==============================] - 13s 102ms/step - loss: 0.9569 - accuracy: 0.5335 - val_loss: 0.7794 - val_accuracy: 0.5526
Epoch 2/200
90/90 [==============================] - ETA: 0s - loss: 0.9077 - accuracy: 0.5156
Epoch 2: val_loss improved from 0.77938 to 0.72181, saving model to ./model/02-0.7218.hdf5
90/90 [==============================] - 8s 94ms/step - loss: 0.9077 - accuracy: 0.5156 - val_loss: 0.7218 - val_accuracy: 0.6118
Epoch 3/200
90/90 [==============================] - ETA: 0s - loss: 0.7625 - accuracy: 0.6205
Epoch 3: val_loss did not improve from 0.72181
90/90 [==============================] - 8s 89ms/step - loss: 0.7625 - accuracy: 0.6205 - val_loss: 0.7429 - val_accuracy: 0.6118
Epoch 4/200
90/90 [==============================] - ETA: 0s - loss: 0.7359 - accuracy: 0.5871
Epo

In [ ]:
# 모델 저장하기
MIX_MODEL_DIR = './model/mixed_data'
if not os.path.exists(MIX_MODEL_DIR):
    os.mkdir(MIX_MODEL_DIR)

#__ 부분: 본인 데이터 형식 적기
model1.save('mixed_data_mobilenetBasic_img_aug_cnn.h5')